In [1]:
import pandas as pd

## Подготовка данных

In [2]:
df: pd.DataFrame = pd.DataFrame()

def reset_df():
    global df
    df = pd.read_csv('data/salaries.csv')
    df.drop(columns=['median', 'modal', 'parsed_from'], inplace=True)
    df.dropna(inplace=True)
    df['predicted'] = False
    
    df = df[df['year'] != 2024]
    
    month_mapping = {
        "январь": 1,
        "февраль": 2,
        "март": 3,
        "апрель": 4,
        "май": 5,
        "июнь": 6,
        "июль": 7,
        "август": 8,
        "сентябрь": 9,
        "октябрь": 10,
        "ноябрь": 11,
        "декабрь": 12
    }
    min_year_by_region = df.groupby('region')['year'].min().reset_index()
    regions = df["region"].unique()
    
    def get_min_year(region: str) -> int:
        return min_year_by_region[min_year_by_region['region'] == region]['year'].values[0]
    
    def get_month_number(month: str) -> int:
        return month_mapping[month]
    
    def create_period(row) -> int:
        min_year = get_min_year(row["region"])
        month_number = get_month_number(row["month"])
        year = row["year"]
        
        return (year - min_year) * 12 + month_number
    
    df['period'] = df.apply(create_period, axis=1)

In [3]:
reset_df()

In [4]:
df

,year,month,region,mean,predicted,period
3,2023,февраль,омск,51550.0,False,74
4,2023,апрель,воронеж,54240.0,False,76
6,2023,апрель,омск,55182.0,False,76
7,2022,февраль,воронеж,45625.0,False,62
9,2022,февраль,омск,44992.0,False,62
...,...,...,...,...,...,...
2683,2019,август,донецк,33594.0,False,32
2684,2017,апрель,донецк,23127.0,False,4
2685,2019,июнь,ростов-на-дону,35343.0,False,30
2686,2019,октябрь,донецк,31141.0,False,34


In [5]:
month_mapping = {
    "январь": 1,
    "февраль": 2,
    "март": 3,
    "апрель": 4,
    "май": 5,
    "июнь": 6,
    "июль": 7,
    "август": 8,
    "сентябрь": 9,
    "октябрь": 10,
    "ноябрь": 11,
    "декабрь": 12
}
min_year_by_region = df.groupby('region')['year'].min().reset_index()
regions = df["region"].unique()

def get_min_year(region: str) -> int:
    return min_year_by_region[min_year_by_region['region'] == region]['year'].values[0]

def get_month_number(month: str) -> int:
    return month_mapping[month]

def create_period(row) -> int:
    min_year = get_min_year(row["region"])
    month_number = get_month_number(row["month"])
    year = row["year"]
    
    return (year - min_year) * 12 + month_number

df['period'] = df.apply(create_period, axis=1)

## Задание 1
#### Коэффициенты линейной регрессии для каждого региона
- t - Период
- y - Значение
- n - Количество периодов

In [6]:
data = []

for region in regions:
    region_df = df[df["region"] == region]

    n = region_df.shape[0]
    period_sum_value = region_df['period'].sum()
    value_sum_value = region_df['mean'].sum()
    period_powered_sum_value = (region_df['period'] ** 2).sum()
    period_to_value_sum_value = (region_df['period'] * region_df['mean']).sum()
    a_1 = (
        (n * period_to_value_sum_value - period_sum_value * value_sum_value)
        / (n * period_powered_sum_value - period_sum_value ** 2)
    )
    a_0 = (
        (value_sum_value - a_1 * period_sum_value)
        / n
    )
    data.append([
        region,
        a_1,
        a_0,
    ])


In [7]:
linear_regression_df = pd.DataFrame(data, columns=['region', 'a_1', 'a_0'])
linear_regression_df

,region,a_1,a_0
0,омск,388.594523,22000.887550
1,воронеж,347.888175,25465.169248
2,пермь,356.128126,24164.804647
3,волгоград,390.765141,21864.755307
4,донецк,700.414326,12701.045898
5,саратов,422.926952,21392.497418
6,тольятти,439.710611,16877.382387
7,тюмень,392.054004,26402.490534
8,барнаул,386.914073,19385.806655
9,махачкала,445.392376,20033.835915


#### Предсказание зарплаты для каждого региона на n периодов

In [8]:
def get_linear_regression_prediction(region: str, period: int) -> float:
    row = linear_regression_df[linear_regression_df['region'] == region].iloc[0]
    return row['a_1'] * period + row['a_0']


In [9]:
future_periods = 24

for region in regions:
    region_df = df[df["region"] == region]
    last_period = region_df['period'].max()
    last_period_row = region_df[region_df['period'] == last_period].iloc[0]

    for i in range(1, future_periods + 1):
        new_month_number = get_month_number(last_period_row["month"]) + i
        new_year = last_period_row["year"] + new_month_number // 12
        new_month = list(month_mapping.keys())[new_month_number % 12]
        new_period = create_period(dict(region=last_period_row["region"], year=new_year, month=new_month))
        mean = get_linear_regression_prediction(region, new_period)
        

        new_row = dict(
            region=region,
            year=new_year,
            month=new_month,
            period=new_period,
            mean=mean,
            predicted=True
        )
        
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
            

In [10]:
from matplotlib import pyplot as plt
from matplotlib import dates
import seaborn as sns

from datetime import date

In [ ]:
for region in regions:
    region_df = df[df["region"] == region]
    region_df = region_df.sort_values(by='period')

    region_df['date'] = region_df.apply(lambda row: date(row['year'], get_month_number(row['month']), 1), axis=1)
    region_df['predicted_value'] = region_df.apply(lambda row: get_linear_regression_prediction(region, row['period']), axis=1)

    plt.figure(figsize=(16, 8))

    real_data = region_df[region_df['predicted'] == False]
    predicted_data = region_df[region_df['predicted'] == True]

    sns.lineplot(
        data=real_data,
        x='date',
        y='mean',
        marker='o',
        markersize=5,
        linestyle='',
        color='blue',
        label='Реальные данные',
    )
    sns.lineplot(
        data=predicted_data,
        x='date',
        y='mean',
        marker='D',
        markersize=5,
        linestyle='',
        color='red',
        label='Прогноз',
    )
    sns.lineplot(
        data=region_df,
        x='date',
        y='predicted_value',
        color='lime',
        linewidth=0.8,
        label='Тренд',
    )

    plt.title(f"Зарплаты в регионе {region.capitalize()}", fontsize=18, weight='bold')
    plt.xlabel("Дата", fontsize=14)
    plt.ylabel("Зарплата", fontsize=14)
    
    plt.gca().xaxis.set_major_locator(dates.MonthLocator(interval=6))
    plt.gca().xaxis.set_major_formatter(dates.DateFormatter('%Y-%m'))

    plt.xticks(rotation=45, fontsize=12)
    plt.yticks(range(int(region_df['mean'].min() // 1000 * 1000), int(region_df['mean'].max()) + 5000, 5000), fontsize=12)
    
    plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
    plt.legend(fontsize=12)

    plt.savefig(f"results/linear_regression/{region}.png")

## Задание 2
#### Подсчет цепные и базисные показатели
- Абсолютный прирост
- Относительный прирост
- Темп роста
- Темп прироста

In [12]:
reset_df()

In [13]:
dataframes = {}

for region in regions:
    region_df = df[df["region"] == region]
    region_df = region_df.sort_values(by='period')
    
    region_df['base_growth'] = region_df['mean'] - region_df['mean'].iloc[0]
    region_df['chain_growth'] = region_df['mean'].diff()
    region_df['base_growth_rate'] = region_df['mean'] / region_df['mean'].iloc[0]
    region_df['chain_growth_rate'] = region_df['mean'] / region_df['mean'].shift(1)
    
    dataframes[region] = region_df

In [14]:
test = dataframes["красноярск"]
test

,year,month,region,mean,predicted,period,base_growth,chain_growth,base_growth_rate,chain_growth_rate
2287,2017,январь,красноярск,33522.0,False,1,0.0,NaN,1.000000,NaN
2581,2017,февраль,красноярск,32532.0,False,2,-990.0,-990.0,0.970467,0.970467
2288,2017,март,красноярск,33181.0,False,3,-341.0,649.0,0.989828,1.019950
2571,2017,апрель,красноярск,33424.0,False,4,-98.0,243.0,0.997077,1.007323
2289,2017,май,красноярск,33443.0,False,5,-79.0,19.0,0.997643,1.000568
...,...,...,...,...,...,...,...,...,...,...
842,2023,август,красноярск,66255.0,False,80,32733.0,1636.0,1.976463,1.025318
558,2023,сентябрь,красноярск,66676.0,False,81,33154.0,421.0,1.989022,1.006354
838,2023,октябрь,красноярск,65382.0,False,82,31860.0,-1294.0,1.950421,0.980593
565,2023,ноябрь,красноярск,63101.0,False,83,29579.0,-2281.0,1.882376,0.965113


## Задание 3
#### Прогнозирование с методом экспоненциального сглаживания Хольта-Уинтерса

In [15]:
reset_df()

In [16]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
future_periods = 24

for region in regions:
    region_df = df[df["region"] == region]
    region_df = region_df.sort_values(by='period')
    last_period = region_df['period'].max()
    last_period_row = region_df[region_df['period'] == last_period].iloc[0]
    
    model = ExponentialSmoothing(region_df['mean'], trend='add', seasonal='add', seasonal_periods=12)
    fit = model.fit()
    forecast = fit.forecast(future_periods)

    for i, mean in enumerate(forecast, 1):
        new_month_number = get_month_number(last_period_row["month"]) + i
        new_year = last_period_row["year"] + new_month_number // 12
        new_month = list(month_mapping.keys())[new_month_number % 12]
        new_period = create_period(dict(region=last_period_row["region"], year=new_year, month=new_month))

        new_row = dict(
            region=region,
            year=new_year,
            month=new_month,
            period=new_period,
            mean=mean,
            predicted=True
        )

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
            

In [ ]:
for region in regions:
    region_df = df[df["region"] == region]
    region_df = region_df.sort_values(by='period')

    region_df['date'] = region_df.apply(lambda row: date(row['year'], get_month_number(row['month']), 1), axis=1)

    plt.figure(figsize=(16, 8))

    real_data = region_df[region_df['predicted'] == False]
    predicted_data = region_df[region_df['predicted'] == True]

    sns.lineplot(
        data=real_data,
        x='date',
        y='mean',
        marker='o',
        markersize=5,
        linestyle='',
        color='blue',
        label='Реальные данные',
    )
    sns.lineplot(
        data=predicted_data,
        x='date',
        y='mean',
        marker='D',
        markersize=5,
        linestyle='',
        color='red',
        label='Прогноз',
    )

    plt.title(f"Зарплаты в регионе {region.capitalize()}", fontsize=18, weight='bold')
    plt.xlabel("Дата", fontsize=14)
    plt.ylabel("Зарплата", fontsize=14)
    
    plt.gca().xaxis.set_major_locator(dates.MonthLocator(interval=6))
    plt.gca().xaxis.set_major_formatter(dates.DateFormatter('%Y-%m'))

    plt.xticks(rotation=45, fontsize=12)
    plt.yticks(range(int(region_df['mean'].min() // 1000 * 1000), int(region_df['mean'].max()) + 5000, 5000), fontsize=12)
    
    plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
    plt.legend(fontsize=12)

    plt.savefig(f"results/exponential_smoothing/{region}.png")